## 2x2 Vectors Graph

A simple model with two input layers and two output layers.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf
import os

In [10]:
# construct the graph

X = tf.placeholder(tf.float32, shape=(4,4), name="input_X")
Y = tf.placeholder(tf.float32, shape=(4,4), name="input_Y")

W_in = [[1,0,1,0], [0,1,0,1], [1,0,1,0], [0,1,0,1]]
B_in = [[0,1,0,1], [1,0,1,0], [0,1,0,1], [1,0,1,0]]

W = tf.Variable(W_in, dtype=tf.float32, name="W")
B = tf.Variable(B_in, dtype=tf.float32, name="B")

XW = tf.matmul(X, tf.transpose(W))
YB = tf.matmul(Y, tf.transpose(B))

Z = tf.multiply( XW, YB, name="output_Z" )
S = tf.add( XW, YB, name="output_S" )

# execute the graph

init = tf.global_variables_initializer()
saver = tf.train.Saver()

X_in = [[1,2,3,4], [10,20,30,40], [100,200,300,400], [1000,2000,3000,4000]]
Y_in = [[5,6,7,8], [50,60,70,80], [500,600,700,800], [5000,6000,7000,8000]]

with tf.Session() as sess:
    sess.run(init)
    Z_eval, S_eval = sess.run([Z, S], feed_dict={X: X_in, Y: Y_in})
    
    # saver.save(sess, "./saved_checkpoints/model.ckpt")
    # tf.train.write_graph(sess.graph, './saved_graph', 'graph.pb', as_text=False)
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input_X": X, "input_Y": Y}, outputs={"output_Z": Z, "output_S": S})
    
    print(Z_eval)
    print(S_eval)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[[5.6e+01 7.2e+01 5.6e+01 7.2e+01]
 [5.6e+03 7.2e+03 5.6e+03 7.2e+03]
 [5.6e+05 7.2e+05 5.6e+05 7.2e+05]
 [5.6e+07 7.2e+07 5.6e+07 7.2e+07]]
[[   18.    18.    18.    18.]
 [  180.   180.   180.   180.]
 [ 1800.  1800.  1800.  1800.]
 [18000. 18000. 18000. 18000.]]


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output_Z,output_S
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input_X,input_Y \
	--input_shapes=4,4:4,4 \
	--output_arrays=output_Z,output_S
```